In [ ]:
import cv2
import numpy as np
import math
import os
from ultralytics import YOLO
from tensorflow.keras.models import load_model


ACCIDENT_VIDEOS_DIR = r"F:\a"
NONACCIDENT_VIDEOS_DIR = r"F:\New folder"
OUTPUT_DATASET_DIR = r"F:\output"

ACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'accident')
NONACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'nonaccident')
os.makedirs(ACCIDENT_OUTPUT_DIR, exist_ok=True)
os.makedirs(NONACCIDENT_OUTPUT_DIR, exist_ok=True)


FRAME_SKIP = 4


yolo_model = YOLO("yolov5l.pt")
vehicle_classes = ["car", "truck", "bus", "motorcycle"]


def are_boxes_close(box1, box2, image_width, image_height, rel_distance_threshold=0.05):

    x1, y1, x2, y2 = box1
    x1b, y1b, x2b, y2b = box2

    
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((x1b + x2b) / 2, (y1b + y2b) / 2)

    
    w1, h1 = x2 - x1, y2 - y1
    w2, h2 = x2b - x1b, y2b - y1b

    
    if (x1 < x2b and x2 > x1b and y1 < y2b and y2 > y1b):
        return True, 0

    
    distance = math.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
    avg_size = (w1 + h1 + w2 + h2) / 4
    normalized_distance = distance / avg_size

    return normalized_distance < 2.0, normalized_distance

def combine_boxes(boxes, padding=0.1):
    
    x1 = min(b[0] for b in boxes)
    y1 = min(b[1] for b in boxes)
    x2 = max(b[2] for b in boxes)
    y2 = max(b[3] for b in boxes)
    
    width, height = x2 - x1, y2 - y1
    pad_x, pad_y = int(width * padding), int(height * padding)
    
    return (max(0, x1 - pad_x), max(0, y1 - pad_y),
            x2 + pad_x, y2 + pad_y)

def expand_roi_to_square(x1, y1, x2, y2, image_width, image_height):
    
    width = x2 - x1
    height = y2 - y1
    
    if width > height:
        diff = width - height
        new_y1 = max(0, y1 - diff // 2)
        new_y2 = y2 + diff - (y1 - new_y1)
        if new_y2 > image_height:
            new_y2 = image_height
            new_y1 = max(0, new_y2 - width)
        new_x1, new_x2 = x1, x2
    elif height > width:
        diff = height - width
        new_x1 = max(0, x1 - diff // 2)
        new_x2 = x2 + diff - (x1 - new_x1)
        if new_x2 > image_width:
            new_x2 = image_width
            new_x1 = max(0, new_x2 - height)
        new_y1, new_y2 = y1, y2
    else:
        new_x1, new_y1, new_x2, new_y2 = x1, y1, x2, y2
    
    return new_x1, new_y1, new_x2, new_y2


def process_video(video_path, output_folder, label):
 
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        return

    frame_count = 0
    roi_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            continue

        height, width, _ = frame.shape
        frame_clean = frame.copy()

        
        results = yolo_model(frame)

        
        vehicle_boxes = []
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = yolo_model.names[class_id]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                if class_name in vehicle_classes and confidence > 0.5:
                    vehicle_boxes.append((x1, y1, x2, y2, class_name, confidence))

        
        processed_vehicles = set()
        accident_rois = []
        for i, box1 in enumerate(vehicle_boxes):
            if i in processed_vehicles:
                continue
            x1, y1, x2, y2, class_name1, conf1 = box1
            close_boxes = [box1[:4]]
            processed_vehicles.add(i)

            for j, box2 in enumerate(vehicle_boxes[i+1:], start=i+1):
                if j in processed_vehicles:
                    continue
                is_close, _ = are_boxes_close(box1[:4], box2[:4], width, height)
                if is_close:
                    close_boxes.append(box2[:4])
                    processed_vehicles.add(j)

            if len(close_boxes) > 1:
                combined_roi = combine_boxes(close_boxes, padding=0.15)
                accident_rois.append(combined_roi)

        
        for roi in accident_rois:
            x1, y1, x2, y2 = roi
            
            new_x1, new_y1, new_x2, new_y2 = expand_roi_to_square(x1, y1, x2, y2, width, height)
            
            square_roi = frame_clean[new_y1:new_y2, new_x1:new_x2]
            
            resized_roi = cv2.resize(square_roi, (224, 224))
            
            roi_count += 1
            output_path = os.path.join(output_folder, f"{label}_frame{frame_count:04d}_roi{roi_count:03d}.jpg")
            cv2.imwrite(output_path, resized_roi)
            print(f"Saved {label} ROI: {output_path}")

    cap.release()
    print(f"Finished processing video: {video_path}")


def process_all_videos():
    
    for filename in os.listdir(ACCIDENT_VIDEOS_DIR):
        if filename.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(ACCIDENT_VIDEOS_DIR, filename)
            process_video(video_path, ACCIDENT_OUTPUT_DIR, "accident")
    
    
    for filename in os.listdir(NONACCIDENT_VIDEOS_DIR):
        if filename.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(NONACCIDENT_VIDEOS_DIR, filename)
            process_video(video_path, NONACCIDENT_OUTPUT_DIR, "nonaccident")


process_all_videos()


PRO TIP  Replace 'model=yolov5l.pt' with new 'model=yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 384x640 3 cars, 531.2ms
Speed: 11.0ms preprocess, 531.2ms inference, 14.0ms postprocess per image at shape (1, 3, 384, 640)
Saved accident ROI: F:\output\accident\accident_frame0004_roi001.jpg

0: 384x640 3 cars, 393.5ms
Speed: 4.0ms preprocess, 393.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 362.6ms
Speed: 3.0ms preprocess, 362.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 335.4ms
Speed: 3.0ms preprocess, 335.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Saved accident ROI: F:\output\accident\accident_frame0016_roi002.jpg

0: 384x640 3 cars, 446.1ms
Speed: 6.0ms preprocess, 446.1ms inference, 1.0ms postprocess per image 

In [1]:
import cv2
import numpy as np
import math
import os
from ultralytics import YOLO
from tensorflow.keras.models import load_model


ACCIDENT_VIDEOS_DIR = r"F:\pratice_dataset\practise_accident_video"
NONACCIDENT_VIDEOS_DIR = r"F:\pratice_dataset\pratice_nono_accident_video"
OUTPUT_DATASET_DIR = r"F:\output"

ACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'accident')
NONACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'nonaccident')
os.makedirs(ACCIDENT_OUTPUT_DIR, exist_ok=True)
os.makedirs(NONACCIDENT_OUTPUT_DIR, exist_ok=True)


FRAME_SKIP = 4


yolo_model = YOLO("yolov5l.pt")
vehicle_classes = ["car", "truck", "bus", "motorcycle"]


def are_boxes_close(box1, box2, image_width, image_height, rel_distance_threshold=0.05):

    x1, y1, x2, y2 = box1
    x1b, y1b, x2b, y2b = box2

    
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((x1b + x2b) / 2, (y1b + y2b) / 2)

    
    w1, h1 = x2 - x1, y2 - y1
    w2, h2 = x2b - x1b, y2b - y1b

    
    if (x1 < x2b and x2 > x1b and y1 < y2b and y2 > y1b):
        return True, 0

    
    distance = math.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
    avg_size = (w1 + h1 + w2 + h2) / 4
    normalized_distance = distance / avg_size

    return normalized_distance < 2.0, normalized_distance

def combine_boxes(boxes, padding=0.1):
    
    x1 = min(b[0] for b in boxes)
    y1 = min(b[1] for b in boxes)
    x2 = max(b[2] for b in boxes)
    y2 = max(b[3] for b in boxes)
    
    width, height = x2 - x1, y2 - y1
    pad_x, pad_y = int(width * padding), int(height * padding)
    
    return (max(0, x1 - pad_x), max(0, y1 - pad_y),
            x2 + pad_x, y2 + pad_y)


def extract_fixed_size_roi(x1, y1, x2, y2, image_width, image_height, output_size=224):
    # Compute the center of the combined ROI
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    
    half_size = output_size // 2
    new_x1 = center_x - half_size
    new_y1 = center_y - half_size
    new_x2 = center_x + half_size
    new_y2 = center_y + half_size

    # Adjust coordinates if they go beyond image boundaries
    if new_x1 < 0:
        new_x1 = 0
        new_x2 = output_size
    if new_y1 < 0:
        new_y1 = 0
        new_y2 = output_size
    if new_x2 > image_width:
        new_x2 = image_width
        new_x1 = image_width - output_size
    if new_y2 > image_height:
        new_y2 = image_height
        new_y1 = image_height - output_size
    
    # Final check to make sure the ROI is the correct size
    # (Note: When near the image edges, the ROI might be less than output_size,
    # adjust as needed or warn)
    return new_x1, new_y1, new_x2, new_y2

def process_video(video_path, output_folder, label):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        return

    frame_count = 0
    roi_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % FRAME_SKIP != 0:
            continue

        height, width, _ = frame.shape
        frame_clean = frame.copy()

        # Run YOLO detection on the frame
        results = yolo_model(frame)

        vehicle_boxes = []
        for result in results:
            for box in result.boxes:
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = yolo_model.names[class_id]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                if class_name in vehicle_classes and confidence > 0.5:
                    vehicle_boxes.append((x1, y1, x2, y2, class_name, confidence))

        processed_vehicles = set()
        accident_rois = []
        for i, box1 in enumerate(vehicle_boxes):
            if i in processed_vehicles:
                continue
            x1, y1, x2, y2, class_name1, conf1 = box1
            close_boxes = [box1[:4]]
            processed_vehicles.add(i)

            for j, box2 in enumerate(vehicle_boxes[i+1:], start=i+1):
                if j in processed_vehicles:
                    continue
                is_close, _ = are_boxes_close(box1[:4], box2[:4], width, height)
                if is_close:
                    close_boxes.append(box2[:4])
                    processed_vehicles.add(j)

            if len(close_boxes) > 1:
                combined_roi = combine_boxes(close_boxes, padding=0.15)
                accident_rois.append(combined_roi)

        for roi in accident_rois:
            x1, y1, x2, y2 = roi

            # Instead of expanding to square and resizing, extract a 224x224 patch directly.
            new_x1, new_y1, new_x2, new_y2 = extract_fixed_size_roi(x1, y1, x2, y2, width, height, output_size=224)

            fixed_roi = frame_clean[new_y1:new_y2, new_x1:new_x2]
            
            # No cv2.resize here; fixed_roi will already be 224x224 (provided the image is large enough)
            roi_count += 1
            output_path = os.path.join(output_folder, f"{label}_frame{frame_count:04d}_roi{roi_count:03d}.jpg")
            cv2.imwrite(output_path, fixed_roi)
            print(f"Saved {label} ROI: {output_path}")

    cap.release()
    print(f"Finished processing video: {video_path}")

def process_all_videos():
    for filename in os.listdir(ACCIDENT_VIDEOS_DIR):
        if filename.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(ACCIDENT_VIDEOS_DIR, filename)
            process_video(video_path, ACCIDENT_OUTPUT_DIR, "accident")

    for filename in os.listdir(NONACCIDENT_VIDEOS_DIR):
        if filename.lower().endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(NONACCIDENT_VIDEOS_DIR, filename)
            process_video(video_path, NONACCIDENT_OUTPUT_DIR, "nonaccident")

process_all_videos()


PRO TIP  Replace 'model=yolov5l.pt' with new 'model=yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|████████████████████████████████████████████████████████████████████████████████| 102M/102M [02:05<00:00, 849kB/s]



0: 512x640 8 cars, 2 trucks, 5 traffic lights, 663.7ms
Speed: 14.5ms preprocess, 663.7ms inference, 22.0ms postprocess per image at shape (1, 3, 512, 640)
Saved accident ROI: F:\output\accident\accident_frame0004_roi001.jpg
Saved accident ROI: F:\output\accident\accident_frame0004_roi002.jpg
Saved accident ROI: F:\output\accident\accident_frame0004_roi003.jpg

0: 512x640 7 cars, 2 trucks, 5 traffic lights, 520.1ms
Speed: 3.0ms preprocess, 520.1ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
Saved accident ROI: F:\output\accident\accident_frame0008_roi004.jpg
Saved accident ROI: F:\output\accident\accident_frame0008_roi005.jpg
Saved accident ROI: F:\output\accident\accident_frame0008_roi006.jpg

0: 512x640 7 cars, 2 trucks, 5 traffic lights, 664.8ms
Speed: 7.0ms preprocess, 664.8ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
Saved accident ROI: F:\output\accident\accident_frame0012_roi007.jpg
Saved accident ROI: F:\output\accident\accident_frame

In [11]:
import os
import math
import cv2
from ultralytics import YOLO

# Directories
ACCIDENT_VIDEOS_DIR = r"F:\pratice_dataset\practise_accident_video"
NONACCIDENT_VIDEOS_DIR = r"F:\pratice_dataset\pratice_nono_accident_video"
OUTPUT_DATASET_DIR = r"F:\output"
ACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'accident')
NONACCIDENT_OUTPUT_DIR = os.path.join(OUTPUT_DATASET_DIR, 'nonaccident')
os.makedirs(ACCIDENT_OUTPUT_DIR, exist_ok=True)
os.makedirs(NONACCIDENT_OUTPUT_DIR, exist_ok=True)

# Frame skipping
FRAME_SKIP = 4

# Load YOLO model and define vehicle classes
yolo_model = YOLO("yolov5l.pt")
vehicle_classes = ["car", "truck", "bus", "motorcycle"]


def are_boxes_close(box1, box2, image_width, image_height, rel_distance_threshold=0.05):
    x1, y1, x2, y2 = box1
    xb1, yb1, xb2, yb2 = box2
    # overlap
    if x1 < xb2 and x2 > xb1 and y1 < yb2 and y2 > yb1:
        return True, 0
    # center distance
    c1 = ((x1+x2)/2, (y1+y2)/2)
    c2 = ((xb1+xb2)/2, (yb1+yb2)/2)
    dist = math.hypot(c1[0]-c2[0], c1[1]-c2[1])
    w1, h1 = x2-x1, y2-y1
    w2, h2 = xb2-xb1, yb2-yb1
    avg = (w1+h1+w2+h2)/4
    norm = dist/avg if avg>0 else float('inf')
    return norm < 2.0, norm


def combine_boxes(boxes, padding=0.1):
    x1 = min(b[0] for b in boxes)
    y1 = min(b[1] for b in boxes)
    x2 = max(b[2] for b in boxes)
    y2 = max(b[3] for b in boxes)
    w, h = x2-x1, y2-y1
    pad_x, pad_y = int(w*padding), int(h*padding)
    return (
        max(0, x1-pad_x),
        max(0, y1-pad_y),
        x2+pad_x,
        y2+pad_y
    )


def expand_roi_to_square(x1, y1, x2, y2, img_w, img_h):
    w = x2 - x1
    h = y2 - y1
    if w > h:
        diff = w - h
        y1n = max(0, y1 - diff//2)
        y2n = y2 + diff - (y1 - y1n)
        if y2n > img_h:
            y2n = img_h
            y1n = max(0, y2n - w)
        x1n, x2n = x1, x2
    elif h > w:
        diff = h - w
        x1n = max(0, x1 - diff//2)
        x2n = x2 + diff - (x1 - x1n)
        if x2n > img_w:
            x2n = img_w
            x1n = max(0, x2n - h)
        y1n, y2n = y1, y2
    else:
        x1n, y1n, x2n, y2n = x1, y1, x2, y2
    return x1n, y1n, x2n, y2n


def get_roi_patch(frame, x1, y1, x2, y2, output_size=224):
    img_h, img_w = frame.shape[:2]
    roi_w, roi_h = x2-x1, y2-y1
    # Large ROI: make square then resize
    if roi_w >= output_size and roi_h >= output_size:
        sx1, sy1, sx2, sy2 = expand_roi_to_square(x1, y1, x2, y2, img_w, img_h)
        patch = frame[sy1:sy2, sx1:sx2]
        patch = cv2.resize(patch, (output_size, output_size))
    else:
        # Small ROI: centered patch of fixed size
        cx = int((x1 + x2)/2)
        cy = int((y1 + y2)/2)
        half = output_size // 2
        nx1 = cx - half
        ny1 = cy - half
        nx2 = nx1 + output_size
        ny2 = ny1 + output_size
        # clamp
        if nx1 < 0:
            nx1, nx2 = 0, output_size
        if ny1 < 0:
            ny1, ny2 = 0, output_size
        if nx2 > img_w:
            nx2, nx1 = img_w, img_w - output_size
        if ny2 > img_h:
            ny2, ny1 = img_h, img_h - output_size
        patch = frame[ny1:ny2, nx1:nx2]
    return patch


def process_video(video_path, output_folder, label):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Could not open video: {video_path}")
        return
    fc = 0
    rc = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        fc += 1
        if fc % FRAME_SKIP != 0:
            continue
        h, w = frame.shape[:2]
        results = yolo_model(frame)
        boxes = []
        for r in results:
            for b in r.boxes:
                cls = int(b.cls[0]); conf = float(b.conf[0])
                name = yolo_model.names[cls]
                x1, y1, x2, y2 = map(int, b.xyxy[0])
                if name in vehicle_classes and conf > 0.5:
                    boxes.append((x1, y1, x2, y2))
        processed = set(); rois = []
        for i, b1 in enumerate(boxes):
            if i in processed: continue
            group = [b1]; processed.add(i)
            for j, b2 in enumerate(boxes[i+1:], start=i+1):
                if j in processed: continue
                close, _ = are_boxes_close(b1, b2, w, h)
                if close:
                    group.append(b2); processed.add(j)
            if len(group) > 1:
                rois.append(combine_boxes(group, padding=0.15))
        for roi in rois:
            x1, y1, x2, y2 = roi
            patch = get_roi_patch(frame, x1, y1, x2, y2)
            rc += 1
            out = os.path.join(output_folder, f"{label}_frame{fc:04d}_roi{rc:03d}.jpg")
            cv2.imwrite(out, patch)
            print(f"Saved {label} ROI: {out}")
    cap.release()
    print(f"Finished: {video_path}")

# Start processing
process_video_paths = [
    (ACCIDENT_VIDEOS_DIR, ACCIDENT_OUTPUT_DIR, 'accident'),
    (NONACCIDENT_VIDEOS_DIR, NONACCIDENT_OUTPUT_DIR, 'nonaccident')
]
for inp, outp, lbl in process_video_paths:
    for fn in os.listdir(inp):
        if fn.lower().endswith(('.mp4', '.avi', '.mov')):
            process_video(os.path.join(inp, fn), outp, lbl)


PRO TIP  Replace 'model=yolov5l.pt' with new 'model=yolov5lu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 512x640 8 cars, 2 trucks, 5 traffic lights, 484.9ms
Speed: 4.0ms preprocess, 484.9ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
Saved accident ROI: F:\output\accident\accident_frame0004_roi001.jpg
Saved accident ROI: F:\output\accident\accident_frame0004_roi002.jpg
Saved accident ROI: F:\output\accident\accident_frame0004_roi003.jpg

0: 512x640 7 cars, 2 trucks, 5 traffic lights, 475.3ms
Speed: 3.0ms preprocess, 475.3ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
Saved accident ROI: F:\output\accident\accident_frame0008_roi004.jpg
Saved accident ROI: F:\output\accident\accident_frame0008_roi005.jpg
Saved accident ROI: F:\output\accident\accident_frame0008_roi006.jpg

0: 512x640 7 cars, 2 